<a href="https://colab.research.google.com/github/Sion1225/Study-Deeplearning-NLP/blob/master/Character%20Level%20Neural%20Machine%20Translation%20with%20Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
"""
import os
import shutil
import zipfile
"""

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [4]:
"""
http = urllib3.PoolManager()
url ='http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)
"""

"\nhttp = urllib3.PoolManager()\nurl ='http://www.manythings.org/anki/fra-eng.zip'\nfilename = 'fra-eng.zip'\npath = os.getcwd()\nzipfilename = os.path.join(path, filename)\nwith http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       \n    shutil.copyfileobj(r, out_file)\n\nwith zipfile.ZipFile(zipfilename, 'r') as zip_ref:\n    zip_ref.extractall(path)\n"

In [5]:
lines = pd.read_csv("/content/drive/MyDrive/GitHub/Study-Deeplearning-NLP/DataSet/spa.txt", names=["src", "tar", "lic"], sep='\t')
del lines["lic"]
print("Count of all semples : ", len(lines))

Count of all semples :  139013


In [7]:
print(lines[:5])

   src      tar
0  Go.      Ve.
1  Go.    Vete.
2  Go.    Vaya.
3  Go.  Váyase.
4  Hi.    Hola.


In [9]:
lines.sample(10)

,src,tar
28858,I can understand Tom.,Puedo entender a Tom.
57807,He comes here once a month.,Él viene aquí una vez al mes.
55907,Tom doesn't speak to Mary.,Tom no está hablando con Mary.
73738,It looks like this car is his.,Parece que este auto es suyo.
31491,Tom saw Mary smiling.,Tom vio a Mary soriendo.
69205,It's been snowing on and off.,Nieva intermitentemente.
29929,It may rain tomorrow.,Es posible que llueva mañana.
47878,He became rich overnight.,Se hizo rico de la noche a la mañana.
77169,I can lend you one if you want.,Te puedo prestar uno si quieres.
16088,He's his own boss.,Él es su propio jefe.


In [10]:
lines.tar = lines.tar.apply(lambda x : "<sos>" + x + "<eos>")
lines.sample(10)

,src,tar
70247,This TV is big and expensive.,<sos>Esta televisión es grande y cara.<eos>
83481,"Since I'm here, let me help you.","<sos>Dado que estoy acá, déjenme ayudarlos.<eos>"
57369,Cars are getting expensive.,<sos>Los autos se están poniendo caros.<eos>
95710,The phone rang and Tom answered it.,<sos>El teléfono sonó y Tom contestó.<eos>
39406,I'm going to the dance.,<sos>Voy a ir al baile.<eos>
113447,Tom felt very guilty about what happened.,<sos>Tom se sentía muy culpable por lo sucedid...
31077,This was Tom's chair.,<sos>Esta era la silla de Tom.<eos>
77037,His house is not far from here.,<sos>Su casa no está lejos de aquí.<eos>
56627,What do we need money for?,<sos>¿Para qué necesitamos dinero?<eos>
5423,I see a queen.,<sos>Veo una reina.<eos>


In [11]:
src_vocab = set()
for line in lines.src :
    for char in line :
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [12]:
src_vocab_size = len(src_vocab) + 1
tar_vocab_size = len(tar_vocab) + 1
print("Source sentences' char set :",src_vocab_size)
print("Target sentences' char set :",tar_vocab_size)

Source sentences' char set : 92
Target sentences' char set : 112


In [15]:
print(src_vocab)
print(tar_vocab)

{'?', '6', 'á', '’', '2', '$', 'Y', 'q', 'U', 'n', 'R', 'l', '.', ' ', 'b', 'u', 'L', '(', 'F', 'z', '0', 'B', 'T', 'A', ',', '+', 'W', '"', 's', 'r', 'h', 'ü', '4', '9', '%', 'ê', 'x', '-', 'j', '\xa0', 'M', '/', 'ã', 'H', 'g', '7', 'D', 'é', 'v', 'c', 'w', '°', 'i', 'd', 'Z', 'V', 'f', 'C', '3', 'G', 'o', 'J', ')', 'N', 'k', '5', 'P', 'm', 'p', '1', 'e', 'I', 't', '₂', 'ó', ';', 'O', 'a', '€', 'X', 'S', '‘', '8', 'K', 'y', 'E', "'", 'Q', '!', ':', 'č'}
{'?', '6', 'á', '—', '2', 'Í', '$', '»', '«', '<', 'Y', 'q', 'U', 'n', 'l', ' ', '.', 'R', 'b', 'É', 'u', 'L', '(', '>', '¨', '¡', 'F', 'ś', 'z', '0', 'T', 'B', 'A', ',', '¿', '+', 'W', '"', 's', 'r', 'h', 'ü', '4', '9', '%', 'ê', 'x', 'с', '-', 'j', 'º', 'M', 'Á', '\xa0', '/', 'H', 'g', '7', 'D', 'é', '&', 'v', 'c', 'w', '°', 'i', 'd', 'Z', 'V', 'f', 'C', 'ñ', '3', 'G', 'o', 'J', ')', '´', 'N', 'k', '5', 'P', 'm', 'ú', 'p', '1', 'e', 'I', 't', 'ó', '₂', ';', 'O', 'a', '€', 'X', 'S', 'Ó', '8', 'K', 'y', 'E', 'ª', '\u200b', "'", 'Q', 'í

In [19]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[26:75])
print(tar_vocab[26:75])

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w']
['<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't']
